# Create Data Files for Workshop

- Full year of hourly data for a single station with extra columns removed
- Extract hourly data for a single 24-hour period from all stations and merge into one CSV file `'weather_airports_24hr_snapshot.csv'`
  - Use UTC time to extract data from the same 24-hour period for all stations
  - May 22, 2018

In [1]:
import pandas as pd

## Full Year of Hourly Data for One Station

Load data, remove unnecessary columns, and save to new csv file

In [2]:
station = 'YVR'
year = 2017
datafile = f'data/processed/weather_hourly_{station}_{year}01-{year}12.csv'
savefile = f'data/workshop/weather_hourly_{station}_{year}.csv'

# Load data
print(f'Reading {datafile}')
df_year = pd.read_csv(datafile, index_col=0)

# Remove unnecessary columns
print('Processing')
cols_drop = ['Station ID', 'Station Name', 'Timezone', 'Datetime (UTC)']
df_year = df_year.drop(cols_drop, axis=1)

# Rename some labels
df_year.index.name = 'Datetime'
df_year = df_year.rename(columns={'Stn Press (kPa)' : 'Pressure (kPa)'})

# Save to CSV
print(f'Saving to {savefile}')
df_year.to_csv(savefile)

# Display the first few rows of the DataFrame
df_year.head()

Reading data/processed/weather_hourly_YVR_201701-201712.csv
Processing
Saving to data/workshop/weather_hourly_YVR_2017.csv


,Temp (deg C),Dew Point Temp (deg C),Rel Hum (%),Wind Dir (deg),Wind Spd (km/h),Visibility (km),Pressure (kPa),Hmdx,Wind Chill,Conditions
Datetime,,,,,,,,,,
2017-01-01 00:00:00,1.2,-0.4,89.0,360.0,8.0,19.3,100.54,NaN,NaN,NaN
2017-01-01 01:00:00,0.9,-0.7,89.0,360.0,2.0,24.1,100.55,NaN,NaN,Cloudy
2017-01-01 02:00:00,1.2,0.6,96.0,80.0,4.0,19.3,100.61,NaN,NaN,Cloudy
2017-01-01 03:00:00,0.6,0.2,97.0,360.0,2.0,19.3,100.65,NaN,NaN,Cloudy
2017-01-01 04:00:00,0.6,0.2,97.0,230.0,3.0,19.3,100.65,NaN,NaN,Cloudy


In [3]:
df_year.describe()

,Temp (deg C),Dew Point Temp (deg C),Rel Hum (%),Wind Dir (deg),Wind Spd (km/h),Visibility (km),Pressure (kPa),Hmdx,Wind Chill
count,8758.000000,8758.000000,8758.000000,8745.000000,8758.000000,8758.000000,8758.000000,446.000000,517.000000
mean,10.249441,6.693663,79.785225,170.810749,13.237040,33.137520,101.601111,26.804933,-5.321083
std,6.632640,5.940729,13.019574,96.231337,7.360099,13.237931,0.853132,1.890264,2.546734
min,-8.400000,-15.700000,18.000000,0.000000,0.000000,0.000000,98.630000,25.000000,-13.000000
25%,5.600000,3.400000,71.000000,90.000000,8.000000,24.100000,101.150000,25.000000,-7.000000
50%,9.800000,6.900000,82.000000,130.000000,12.000000,32.200000,101.630000,26.000000,-5.000000
75%,15.400000,11.200000,90.000000,270.000000,17.000000,48.300000,102.120000,28.000000,-4.000000
max,29.200000,19.200000,100.000000,360.000000,68.000000,64.400000,103.940000,33.000000,0.000000


In [4]:
df_year['Conditions'].value_counts(normalize=True)

Cloudy                                          0.256709
Mostly Cloudy                                   0.208062
Mainly Clear                                    0.205436
Clear                                           0.122987
Rain                                            0.113852
Rain Showers                                    0.027064
Fog                                             0.015530
Rain,Fog                                        0.015074
Snow                                            0.009136
Drizzle,Fog                                     0.004568
Moderate Rain,Fog                               0.004111
Moderate Rain                                   0.003540
Drizzle                                         0.002284
Rain,Drizzle,Fog                                0.001713
Snow Showers                                    0.001599
Rain,Snow                                       0.001370
Snow,Fog                                        0.000914
Freezing Fog                   

## 24-hour Snapshot of All Stations

Use UTC date/time to extract the same 24-hour period from each station and merge into a single DataFrame.

In [5]:
def extract_day(datafile, day_utc):
    """Read data from csv file and return DataFrame with subset for the specified UTC day"""
    df_in = pd.read_csv(datafile, index_col='Datetime (UTC)', parse_dates=True)
    
    # Extract data for specifed UTC data
    df = df_in[day_utc]
    
    # Change index to local datetime and shuffle the UTC datetime column over
    df = df.reset_index().set_index('Datetime (Local Standard)')
    columns = df.columns.tolist()
    columns.remove('Datetime (UTC)')
    columns = columns[:3] + ['Datetime (UTC)'] + columns[3:]
    df = df[columns]
    
    # Adjust label for Pressure column
    df = df.rename(columns={'Stn Press (kPa)' : 'Pressure (kPa)'})
    
    # Shorten station names
    df['Station Name'] = df['Station Name'].str.replace(' INTL A', '').str.replace(' A', '')
    
    return df

In [6]:
# Create a list of station codes sorted by station name
stations_info = pd.read_csv('data/airport_stations.csv', index_col=0)
stations = stations_info.sort_values('Name').index.tolist()
print(stations)

['YYC', 'YYG', 'YEG', 'YFC', 'YHZ', 'YFB', 'YUL', 'YOW', 'YXS', 'YQB', 'YQR', 'YXE', 'YYT', 'YQT', 'YYZ', 'YVR', 'YYJ', 'YXY', 'YWG', 'YZF']


In [7]:
# UTC day to extract
day_utc = '2018-05-22'

# File name for saving the merged data
savefile_snapshot = 'data/workshop/weather_airports_24hr_snapshot.csv'

# Extract specified subset for each station and save as a list of DataFrames
df_list = []
for station in stations:
    datafile = f'data/processed/weather_hourly_{station}_201801-201807.csv'
    df = extract_day(datafile, day_utc)
    df_list.append(df)
    
# Concatenate into a single DataFrame
df_snapshot = pd.concat(df_list, axis=0)

# Save to file
print(f'Saving to {savefile_snapshot}')
df_snapshot.to_csv(savefile_snapshot)

# Display a random sampling of 10 rows
df_snapshot.sample(10, random_state=1)

Saving to data/workshop/weather_airports_24hr_snapshot.csv


,Station ID,Station Name,Timezone,Datetime (UTC),Temp (deg C),Dew Point Temp (deg C),Rel Hum (%),Wind Dir (deg),Wind Spd (km/h),Visibility (km),Pressure (kPa),Hmdx,Wind Chill,Conditions
Datetime (Local Standard),,,,,,,,,,,,,,
2018-05-22 01:30:00,YYT,ST. JOHN'S,NST,2018-05-22 05:00:00,5.5,2.3,80.0,270.0,23.0,24.1,98.99,NaN,NaN,Mostly Cloudy
2018-05-22 13:00:00,YXE,SASKATOON,CST,2018-05-22 19:00:00,24.4,6.1,30.0,350.0,7.0,24.1,95.48,NaN,NaN,Mainly Clear
2018-05-22 06:00:00,YYG,CHARLOTTETOWN,AST,2018-05-22 10:00:00,3.1,1.5,89.0,360.0,1.0,24.1,101.64,NaN,NaN,Clear
2018-05-22 12:00:00,YVR,VANCOUVER,PST,2018-05-22 20:00:00,18.2,12.1,68.0,240.0,12.0,48.3,101.31,NaN,NaN,Mainly Clear
2018-05-22 14:00:00,YXY,WHITEHORSE,PST,2018-05-22 22:00:00,12.4,-2.7,35.0,150.0,19.0,64.4,92.32,NaN,NaN,Mostly Cloudy
2018-05-21 19:00:00,YVR,VANCOUVER,PST,2018-05-22 03:00:00,16.8,10.5,66.0,230.0,6.0,48.3,101.52,NaN,NaN,Clear
2018-05-22 15:00:00,YVR,VANCOUVER,PST,2018-05-22 23:00:00,19.4,13.2,67.0,240.0,10.0,48.3,101.11,NaN,NaN,Clear
2018-05-22 11:00:00,YEG,EDMONTON,MST,2018-05-22 18:00:00,24.4,6.0,30.0,120.0,8.0,24.1,93.11,NaN,NaN,Clear
2018-05-22 04:00:00,YQR,REGINA,CST,2018-05-22 10:00:00,13.2,4.9,57.0,210.0,8.0,24.1,94.32,NaN,NaN,Mostly Cloudy
